<a href="https://colab.research.google.com/github/inigmat/exupery/blob/main/CSP_binpacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install binpacking

In [2]:
# Check the csv data
import pandas as pd

CSV_URL = 'https://raw.githubusercontent.com/inigmat/exupery/main/files/bbs.csv'
data = pd.read_csv(CSV_URL)
data.head(5)

,index,Unique Mark,Dia,Length,Qty
0,0,100,10,2350,96
1,1,101,10,2800,24
2,2,102,10,3030,336
3,3,103,10,4230,144
4,4,200,12,450,320


In [3]:
%%time
from binpacking import to_constant_volume

# Constants
BAR_LENGTH = 11700
CSV_URL = CSV_URL


def calculate_required_bars(df: pd.DataFrame, bar_length: int) -> tuple:
    """
    Calculates the required number of reinforcement bars and the cutting schemes.

    Args:
        df (pd.DataFrame): DataFrame containing the length and quantity of bars.
        bar_length (int): Length of the bars.

    Returns:
        tuple: Required number of bars and the cutting schemes.
    """
    items = []
    # Extracting data from DataFrame
    for length, quantity in zip(df['Length'], df['Qty']):
        items.extend([length] * quantity)
    # Calculating the cutting schemes using bin packing algorithm
    cutting_scheme = to_constant_volume(items, bar_length)
    return cutting_scheme


def generate_cutting_table(cutting_data: dict) -> pd.DataFrame:
    """
    Generates a cutting table based on the cutting data.

    Args:
        cutting_data (dict): Dictionary containing cutting schemes for different diameters.

    Returns:
        pd.DataFrame: Cutting table with quantity, cuts, utilization, scrap, and diameter columns.
    """
    cutting_df = pd.DataFrame(columns=["Qty", "Cuts", "Utilization", "Scrap", "Dia"])
    # Processing cutting data for each diameter
    for dia, cutting_scheme in cutting_data.items():
        df_temp = pd.DataFrame(zip(cutting_scheme), columns=["Cuts"])
        # Calculate the quantity of each cutting scheme
        df_temp["Qty"] = df_temp["Cuts"].apply(lambda x: f"{cutting_scheme.count(x)}x")
        # Calculate the total length used by each cutting scheme
        df_temp["Utilization"] = df_temp["Cuts"].apply(sum)
        # Calculate the amount of scrap for each cutting scheme
        df_temp["Scrap"] = BAR_LENGTH - df_temp["Utilization"]
        # Add the diameter column
        df_temp["Dia"] = dia
        # Change the type of values in the Cuts column to drop duplicates
        df_temp["Cuts"] = df_temp["Cuts"].astype(str)
        # Concatenate df_temp with df
        df = pd.concat([data, df_temp], ignore_index=True)
    # Drop duplicate rows
    cutting_df = df.drop_duplicates()
    # Reset the index
    cutting_df = df.reset_index(drop=True)
    return cutting_df


def main() -> pd.DataFrame:
    # Read the data from CSV
    data = pd.read_csv(CSV_URL)
    cutting_data = {}
    # Process data for each unique diameter
    for dia in data['Dia'].unique():
        data_dia = data[data['Dia'] == dia]
        # Calculate the cutting schemes and store them in a dictionary
        cutting_scheme = calculate_required_bars(data_dia, BAR_LENGTH)
        cutting_data[dia] = cutting_scheme
        # Print the required number of reinforcement bars for each diameter
        print(f"Required number of reinforcement bars with diameter {dia} mm: {len(cutting_scheme)} pcs.")
    # Generate the cutting table
    cutting_schema = generate_cutting_table(cutting_data)
    return cutting_schema


if __name__ == "__main__":
    # Execute the main function and store the resulting cutting table
    cutting_schema = main()


Required number of reinforcement bars with diameter 10 mm: 168 pcs.
Required number of reinforcement bars with diameter 12 mm: 138 pcs.
Required number of reinforcement bars with diameter 16 mm: 143 pcs.
Required number of reinforcement bars with diameter 20 mm: 759 pcs.
CPU times: user 449 ms, sys: 7.68 ms, total: 457 ms
Wall time: 510 ms


# Showing the results

In [4]:
#Show the table
cutting_schema

,index,Unique Mark,Dia,Length,Qty,Cuts,Utilization,Scrap
0,0.0,100.0,10,2350.0,96,NaN,NaN,NaN
1,1.0,101.0,10,2800.0,24,NaN,NaN,NaN
2,2.0,102.0,10,3030.0,336,NaN,NaN,NaN
3,3.0,103.0,10,4230.0,144,NaN,NaN,NaN
4,4.0,200.0,12,450.0,320,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
785,NaN,NaN,20,NaN,50x,"[1740, 1740, 1740, 1740, 1740, 1740]",10440.0,1260.0
786,NaN,NaN,20,NaN,50x,"[1740, 1740, 1740, 1740, 1740, 1740]",10440.0,1260.0
787,NaN,NaN,20,NaN,50x,"[1740, 1740, 1740, 1740, 1740, 1740]",10440.0,1260.0
788,NaN,NaN,20,NaN,50x,"[1740, 1740, 1740, 1740, 1740, 1740]",10440.0,1260.0


# Downloading the results

In [5]:
from google.colab import files

# determining the name of the file
file_name = 'cutting_schema.xlsx'

# saving the excel
cutting_schema.to_excel(file_name)

# downloading the file
files.download('cutting_schema.xlsx')
print('DataFrame is written to Excel File successfully. Save the file')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DataFrame is written to Excel File successfully. Save the file
